In [9]:
transacoes = [
    ['Banana', 'Leite'],
    ['Banana', 'Laranja', 'Leite', 'Ovos'],
    ['Banana', 'Laranja', 'Leite'],
    ['Laranja', 'Ovos'],
    ['Banana', 'Laranja', 'Ovos'],
    ['Banana', 'Laranja', 'Leite', 'Ovos'],
    ['Laranja', 'Leite'],
    ['Banana', 'Laranja', 'Leite']
]

### FP-Tree

**A FP-Tree organiza os itens frequentes em caminhos compartilhados**. Se várias transações têm os mesmos itens, eles são armazenados uma única vez na árvore.
- Em vez de guardar todas as transações repetidamente, a FP-Tree armazena apenas o necessário para encontrar os padrões frequentes.
- Isso reduz o uso de memória e acelera o processo de mineração.

In [10]:
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item  # Nome do item
        self.count = count  # Contagem do item
        self.parent = parent  # Nó pai
        self.children = {}  # Nós filhos
        self.next = None  # Ponteiro para o próximo nó com o mesmo item (lista encadeada)

    def increment(self, count):
        self.count += count

# Aqui podemos definir o suporte minimo como a frequencia minima
def construir_fp_tree(transacoes, suporte_minimo):
    
    contagem_itens = {}
    for transacao in transacoes:
        for item in transacao:
            contagem_itens[item] = contagem_itens.get(item, 0) + 1 # se não existe chave, coloca 0 + 1 para iniciar chave

    # Remover itens que não atendem ao suporte mínimo
    itens_frequentes = { item: count for item, count in contagem_itens.items() if count >= suporte_minimo }
    if not itens_frequentes:
        return None, None

    # Ordenar os itens por frequência (decrescente)
    ordem_itens = sorted(itens_frequentes.keys(), key=lambda x: itens_frequentes[x], reverse=True)

    # Construir o FP-Tree
    raiz = FPNode(None, None, None)
    tabela_encadeamento = {}

    for transacao in transacoes:
        
        # Ordena os itens da transação de acordo com a ordem global definida anteriormente
        ordered_item_set = [item for item in transacao if item in itens_frequentes]
        ordered_item_set.sort(key=lambda x: ordem_itens.index(x))

        no_atual = raiz
        for item in ordered_item_set: # Para cada item na transação ordenada pela frequencia...
            
            if item in no_atual.children: # Se o item já está no nó atual
                no_atual.children[item].increment(1) # Incrementa ele no nó atual
            
            else: 
                # Caso o item não esteja cria um novo nó com ele e contagem atual = 1
                novo_no = FPNode(item, 1, no_atual)
                no_atual.children[item] = novo_no

                # Atualiza a tabela de encadeamento
                if item in tabela_encadeamento:
                    no_anterior = tabela_encadeamento[item]
                    while no_anterior.next is not None:
                        no_anterior = no_anterior.next
                    no_anterior.next = novo_no
                else:
                    tabela_encadeamento[item] = novo_no

            # Atualiza o nó atual como sendo o nó que a gente acabou de incrementar
            # Na próxima iteração, vamos ver se o próximo item da lista transação ordenada está
            # nos filhos do nó que a gente acabou de incrementar.
            no_atual = no_atual.children[item]

    return raiz, tabela_encadeamento

In [11]:
def print_fp_tree(node, indent=0):
    if node is None:
        return

    print("  " * indent + f"Item: {node.item}, Count: {node.count}")

    for child_item, child_node in node.children.items():
        print_fp_tree(child_node, indent + 1)

def print_tabela_encadeamento(tabela_encadeamento):
    print("\nTabela de Encadeamento:")
    for item, node in tabela_encadeamento.items():
        print(f"Item: {item}")
        no_atual = node
        while no_atual is not None:
            print(f"  Node: {no_atual.item}, Count: {no_atual.count}")
            no_atual = no_atual.next

fp_tree, tabela_encadeamento = construir_fp_tree(transacoes, 2)
print("FP-Tree:")
print_fp_tree(fp_tree)
print_tabela_encadeamento(tabela_encadeamento)

FP-Tree:
Item: None, Count: None
  Item: Banana, Count: 1
    Item: Leite, Count: 1
  Item: Laranja, Count: 7
    Item: Banana, Count: 5
      Item: Leite, Count: 4
        Item: Ovos, Count: 2
      Item: Ovos, Count: 1
    Item: Ovos, Count: 1
    Item: Leite, Count: 1

Tabela de Encadeamento:
Item: Banana
  Node: Banana, Count: 1
  Node: Banana, Count: 5
Item: Leite
  Node: Leite, Count: 1
  Node: Leite, Count: 4
  Node: Leite, Count: 1
Item: Laranja
  Node: Laranja, Count: 7
Item: Ovos
  Node: Ovos, Count: 2
  Node: Ovos, Count: 1
  Node: Ovos, Count: 1


### Vantagens da FP-Tree

O apriori precisa varrer o dataset várias vezes para calcular o suporte dos candidatos

O FP-Growth varre o dataset **duas vezes apenas**:
1. Conta a frequência dos itens individuais
2. Constrói a FP-Tree

Com a construção da FP-Tree, a mineração é feita diretamente na árvore, sem a necessidade de acessar o dataset novamente.

### Conditional Pattern Base

Agora vamos construir uma Conditional Pattern Base para cada item.

A Conditional Pattern Base é uma **coleção de caminhos na FP-Tree que levam a um item específico**. 
- Esses caminhos representam as transações que contêm o item, mas sem o item em si.

In [24]:
def construir_conditional_pattern_base(tabela_encadeamento, item):
    conditional_pattern_base = []

    # Começa pelo primeiro nó do item na tabela de encadeamento
    no_atual = tabela_encadeamento[item]

    while no_atual is not None:
        # Coleta o caminho do nó atual até a raiz
        caminho = []
        contagem = no_atual.count
        no_pai = no_atual.parent

        while no_pai.item is not None:  # Para antes de chegar na raiz (item None)
            caminho.append(no_pai.item)
            no_pai = no_pai.parent

        # Inverte o caminho para manter a ordem correta
        if caminho:
            caminho.reverse()
            conditional_pattern_base.append((caminho, contagem))

        # Vai para o próximo nó do item na lista encadeada
        no_atual = no_atual.next

    return conditional_pattern_base

cpb_leite = construir_conditional_pattern_base(tabela_encadeamento, "Leite")
cpb_Banana = construir_conditional_pattern_base(tabela_encadeamento, "Banana")

print("Exemplo para Leite:")
print(cpb_leite)

print("Exemplo para Banana:")
print(cpb_Banana)

Exemplo para Leite:
[(['Banana'], 1), (['Laranja', 'Banana'], 4), (['Laranja'], 1)]
Exemplo para Banana:
[(['Laranja'], 5)]


### FP-Tree condicional

Para um item $X$ é uma arvore que:
1. Representa os padrões frequentes associados a $X$
2. Inclui apenas os itens que aparecem em todos os caminhos da **Conditional Pattern Base** de $X$
3. Armazena as contagens acumuladas desses itens, refletindo **quantas vezes eles aparecem nas transações que contêm** $X$

In [ ]:
def construir_fp_tree_condicional(conditional_pattern_base):
    
    # Passo 1: Identificar itens que aparecem em todos os caminhos
    if not conditional_pattern_base:
        return None, None

    # Conjunto de itens comuns (inicialmente, todos os itens do primeiro caminho)
    itens_comuns = set(conditional_pattern_base[0][0])

    for caminho, _ in conditional_pattern_base[1:]:
        itens_comuns.intersection_update(caminho)

    # Construir a FP-Tree Condicional apenas com os itens comuns
    raiz = FPNode(None, None, None)
    tabela_encadeamento = {}

    for caminho, contagem in conditional_pattern_base:
        # Filtra os itens do caminho que estão em itens_comuns
        caminho_filtrado = [item for item in caminho if item in itens_comuns]
        if not caminho_filtrado:
            continue  # Ignora caminhos vazios após a filtragem

        # Insere o caminho filtrado na árvore
        no_atual = raiz
        for item in caminho_filtrado:
            if item in no_atual.children:
                no_atual.children[item].increment(contagem)
            else:
                novo_no = FPNode(item, contagem, no_atual)
                no_atual.children[item] = novo_no

                # Atualiza a tabela de encadeamento
                if item in tabela_encadeamento:
                    no_anterior = tabela_encadeamento[item]
                    while no_anterior.next is not None:
                        no_anterior = no_anterior.next
                    no_anterior.next = novo_no
                else:
                    tabela_encadeamento[item] = novo_no

            no_atual = no_atual.children[item]

    return raiz, tabela_encadeamento

print("Exemplo para Banana:")
print_fp_tree(construir_fp_tree_condicional(construir_conditional_pattern_base(tabela_encadeamento, "Banana"))[0])

Exemplo para Leite:
Item: None, Count: None
  Item: Laranja, Count: 5


In [15]:
def fp_growth(transacoes, suporte_minimo):

    fp_tree, tabela_encadeamento = construir_fp_tree(transacoes, suporte_minimo)

    padroes_frequentes = []
    minerar_padroes(fp_tree, tabela_encadeamento, suporte_minimo, [], padroes_frequentes)

    return padroes_frequentes

def minerar_padroes(fp_tree, tabela_encadeamento, suporte_minimo, prefixo, padroes_frequentes):
    
    # Ordena os itens pela frequência (do menos frequente para o mais frequente)
    itens = sorted(tabela_encadeamento.keys(), key=lambda x: tabela_encadeamento[x].count)

    for item in itens:
        # Cria um novo padrão frequente adicionando o item ao prefixo
        novo_prefixo = prefixo.copy()
        novo_prefixo.append(item)
        padroes_frequentes.append((novo_prefixo, tabela_encadeamento[item].count))

        # Construir a Conditional Pattern Base para o item
        conditional_pattern_base = construir_conditional_pattern_base(tabela_encadeamento, item)

        # Construir a FP-Tree Condicional
        if conditional_pattern_base:
            fp_tree_condicional, tabela_encadeamento_condicional = construir_fp_tree_condicional(conditional_pattern_base)

            # Minerar padrões recursivamente na FP-Tree Condicional
            if tabela_encadeamento_condicional:
                minerar_padroes(fp_tree_condicional, tabela_encadeamento_condicional, suporte_minimo, novo_prefixo, padroes_frequentes)

In [16]:
suporte_minimo = 2
padroes_frequentes = fp_growth(transacoes, suporte_minimo)

print("Padrões Frequentes:")
for padrao, suporte in padroes_frequentes:
    print(f"{padrao}: {suporte}")

Padrões Frequentes:
['Banana']: 1
['Banana', 'Laranja']: 5
['Leite']: 1
['Ovos']: 2
['Ovos', 'Laranja']: 4
['Laranja']: 7


A partir dos padrões frequentes, podemos gerar regras de associação de forma similar ao algoritmo apriori, usando as medidas de confidence, lift ou conviction.

Uma outra alternativa seria não construir a FP-Tree condicional e pegar os conjuntos de cada Conditional Pattern Base e usá-los como itemsets para o algoritmo a priori.
- Note que a frequência de cada itemset desse é representado pela contagem já calculada na etapa do Conditional Pattern Base.
- Além disso, o itemset não vai ser o conjunto e sim o conjunto + o respectivo item.